# Throughput
This notebook creates a throughput report for test results. It ties into the **Test Monitor Service** for retrieving filtered test results, the **Notebook Execution Service** for running outside of Jupyterhub, and the **Test Monitor Reports page** at #testmonitor/reports for displaying results.

The parameters and output use a schema recognized by the Test Monitor Reports page, which can be implemented by various report types. The Throughput notebook produces data that is best shown in a bar graph.

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service. The SystemLink Test Monitor Client provides access to test result data for processing.

In [1]:
import copy
import datetime
import pandas as pd
import scrapbook as sb
from dateutil import tz

import systemlink.clients.nitestmonitor as testmon

### Parameters
- `results_filter`: Dynamic Linq query filter for test results from the Test Monitor Service  
  Options: Any valid Test Monitor Results Dynamic Linq filter  
  Default: `'startedWithin <= "30.0:0:0"'`
- `group_by`: The dimension along which to reduce; what each bar in the output graph represents  
  Options: Day, System, Test Program, Operator, Part Number  
  Default: Day

Parameters are also listed in the metadata for the parameters cell, along with their default values. The Notebook Execution services uses that metadata to pass parameters from the Test Monitor Reports page to this notebook. Available `group_by` options are listed in the metadata as well; the Test Monitor Reports page uses these to validate inputs sent to the notebook.

To see the metadata, select the code cell and click the wrench icon in the far left panel.

In [3]:
results_filter = 'startedWithin <= "30.0:0:0"'
products_filter = ''

### Create Test Monitor client
Establish a connection to SystemLink over HTTP.

In [4]:
results_api = testmon.ResultsApi()

### Query for results
Query the Test Monitor Service for results matching the `results_filter` parameter.

In [6]:
results_query = testmon.ResultsAdvancedQuery(
    results_filter, product_filter=products_filter, order_by=testmon.ResultField.STARTED_AT
)

results = []

response = await results_api.query_results_v2(post_body=results_query)
while response.continuation_token:
    results = results + response.results
    results_query.continuation_token = response.continuation_token
    response = await results_api.query_results_v2(post_body=results_query)
results_list = [result.to_dict() for result in results]
#display(results_list)

### Create pandas dataframe
Put the data into a dataframe whose columns are test result id, start time, and group name.

In [10]:
df_results = pd.DataFrame.from_records(results_list)
df_results = df_results.join(df_results['status'].apply(pd.Series))
df_results['started_at'] = df_results['started_at'].apply(lambda t: t.tz_localize(None))
df_results['updated_at'] = df_results['updated_at'].apply(lambda t: t.tz_localize(None))
df_results.pop('status')
df_results.pop('status_type_summary')
properties = pd.json_normalize(df_results["properties"])
df_results = pd.concat([df_results.drop(["properties"], axis = 1), properties],axis =1 )
display(df_results)

,started_at,updated_at,program_name,id,system_id,host_name,operator,part_number,serial_number,total_time_in_seconds,...,PBS,PBS Lot,Processing Solution,Processing Solution Lot,EQ Number,Firmware Version,Orange Processing Solution,Orange Processing Solution Lot,MKII Puritan Sterile Nasal Swab,MKII Puritan Sterile Nasal Swab Lot
0,2021-05-24 01:03:57.034212,2021-05-24 22:03:41.136830,CTS_eStick_Manual.seq,60ac22b1ca744a9ab41586b5,20RH001LAU--SN-PF26NK02--MAC-F8-75-A4-F2-C6-3D,CEQ0104,Ruchi,M1000843,1262,20.658418,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-24 01:04:03.384143,2021-05-24 22:03:41.213830,CTS_eStick_Manual.seq,60ac22b2ca744a9ab41586bb,20RH001LAU--SN-PF26NK02--MAC-F8-75-A4-F2-C6-3D,CEQ0104,Ruchi,M1000843,1117,12.547906,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-05-24 01:04:39.818724,2021-05-24 22:03:43.193830,CTS_eStick_Manual.seq,60ac22b7ca744a9ab41586e0,20RH001LAU--SN-PF26NK02--MAC-F8-75-A4-F2-C6-3D,CEQ0104,Ruchi,M1000843,905970237,12.437050,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-05-24 01:05:13.842006,2021-05-24 22:03:52.623711,CTS_eStick_Manual.seq,60ac22bdca744a9ab4158702,20RH001LAU--SN-PF26NK02--MAC-F8-75-A4-F2-C6-3D,CEQ0104,Ruchi,M1000843,905970237,20.606943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-05-24 01:05:39.234713,2021-05-24 22:03:52.794712,CTS_eStick_Manual.seq,60ac22beca744a9ab415870b,20RH001LAU--SN-PF26NK02--MAC-F8-75-A4-F2-C6-3D,CEQ0104,Ruchi,M1000843,905970238,20.603963,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7813,2021-06-22 04:25:14.244894,2021-06-22 04:31:11.571845,QC Wet Test System,60d16784ca744a3c64c0710e,20QDA01KAU--SN-PF2HFVDE--MAC-CC-D9-AC-B3-B8-39,,Karissa Mears,M1000226,201343308,0.000000,...,NaN,NaN,NaN,NaN,EQ1014994-1,NaN,NaN,NaN,NaN,NaN
7814,2021-06-22 04:30:49.130124,2021-06-22 04:31:00.917845,QC Wet Test System,60d1677bca744a3c64c0703e,20QDA01KAU--SN-PF2HFVDE--MAC-CC-D9-AC-B3-B8-39,,KM,M1000226,218105575,0.000000,...,NaN,NaN,NaN,NaN,EQ1014994-1,NaN,NaN,NaN,NaN,NaN
7815,2021-06-22 04:31:02.470448,2021-06-22 04:31:21.459718,QC Wet Test System,60d1678fca744a3c64c071e4,20QDA01KAU--SN-PF2HFVDE--MAC-CC-D9-AC-B3-B8-39,,KM,M1000226,134268609,0.000000,...,NaN,NaN,NaN,NaN,EQ1014994-1,NaN,NaN,NaN,NaN,NaN
7816,2021-06-22 04:31:05.094430,2021-06-22 04:31:31.339115,QC Wet Test System,60d16799ca744a3c64c072b4,20QDA01KAU--SN-PF2HFVDE--MAC-CC-D9-AC-B3-B8-39,,KM,M1000226,201343308,0.000000,...,NaN,NaN,NaN,NaN,EQ1014994-1,NaN,NaN,NaN,NaN,NaN


### KPI calculations
SN usage percentage : (latest_SN - Start_SN) / (End_SN - Start_SN) *100

In [12]:
#conver to int
df_SN_Start = pd.to_numeric(df_results['TS SN Start'],errors='ignore')
df_SN_End = pd.to_numeric(df_results['TS SN End'],errors='ignore')
df_SN_Latest = pd.to_numeric(df_results['TS SN Current'],errors='ignore')

#calcultion
df_SN_percentage = (df_SN_Latest-df_SN_Start) / (df_SN_End - df_SN_Start) *100
df_SN_percentage.name='SN_Percentage'

#merge into table
df_results = pd.concat([df_results, df_SN_percentage], axis = 1)

#display(df_results)

### Convert the dataframe to the SystemLink reports output format
The result format for a SystemLink report consists of a list of output objects as defined below:
- `type`: The type of the output. Accepted values are 'data_frame' and 'scalar'.
- `id`: Corresponds to the id specified in the 'output' metadata. Used for returning multiple outputs with the 'V2' report format.
- `data`: A dict representing the 'data_frame' type output data.
    - `columns`: A list of dicts containing the names and data type for each column in the dataframe.
    - `values`: A list of lists containing the dataframe values. The sublists are ordered according to the 'columns' configuration.
- `value`: The value returned for the 'scalar' output type.
- `config`: The configurations for the given output.
    - `title`: The output title.
    - `graph`: The graph configurations.
        - `axis_labels`: The x-axis label and y-axis label.
        - `plots`: A list of plots to display mapped from the dataframe's columns, along with configuration options.
            - `x`: The dataframe column corresponding to the x-axis values.
            - `y`: The dataframe column corresponding to the y-axis values.
            - `style`: The plot's style. Accepted values are ['LINE', 'BAR', 'SCATTER'].
            - `color`: The plot's color. Accepted formats are ['blue', '#0000ff', 'rbg(0,0,255)'].
            - `label`: The plot's name, to be shown in a plot legend. 
            - `secondary_y`: Whether or not to display this plot on a second y-axis.
            - `group_by`: A list of columns in the dataframe on which to group data, e.g. to color individual points.
        - `orientation`: 'HORIZONTAL' or 'VERTICAL'.
        - `stacked`: Whether or not to display the plots stacked on top of each other.

Here is an example of a notebook result with two outputs, one of which is a dataframe with two columns, and the other is a scalar value:
```
[{
    'type': 'data_frame',
    'id': 'output_id_1',
    'data': {
        'columns': [
            {'name': 'time', 'type': 'datetime'},
            {'name': 'value', 'type': 'number'}
         ],
        'values': [
            ['2020-09-29T00:00:00.000Z', 46.1538461538],
            ['2020-09-30T00:00:00.000Z', 63.1578947368],
            ...
         ]
    },
    'config': {
        'title': 'My Title',
        'graph': {
            'axis_labels': ['X Axis', 'Y Axis'],
            'orientation': 'VERTICAL',
            'plots': [
                {'x': 'time', 'y': 'value', 'style': 'BAR', 'color': '#0000ff', 'label': 'Plot 1'}
            ]
        }
    }
}, {
    'type': 'scalar',
    'id': 'output_id_2',
    'config': {
        'title': 'My Title'
    },
    'value': 5
}]
```

For this report, there is one output, which is a dataframe with two columns. For a grouping of 'Day', the first column contains ISO-8601 date strings. For any other grouping option, the first column contains categorical string values. The second column contains numerical values representing the throughput.

| started_at                 | throughput    |
|----------------------------|---------------|
| '2020-09-29'               | 23            |
| '2020-09-30'               | 45            |
| '2020-10-01'               | 30            |

The graph configuration specifies a single plot, where the x-axis is the group values and the y-axis is the throughput. We use Pandas to convert the dataframe built in the previous cells into a tabular format and then return that with the result object.

In [90]:
df_dict = {
    'columns': pd.io.json.build_table_schema(df_results, index=False)['fields'],
    'values': df_results.values.tolist(),
}

results_graph = {
    'type': 'data_frame',
    'id': 'results_graph',
    'data': df_dict
}

result = [results_graph]

### Record results with Scrapbook

In [7]:
sb.glue('result', result)